In [15]:
import torch
import pickle
import librosa
import numpy as np
import soundfile as sf
from pathlib import Path
import IPython.display as ipd

from constants import *
from inferer import Inferer
from plotter import plot_waves
from disk_utils import load_model

In [17]:
with open("dataset/features/min_max.pkl", "rb") as handle:
    min_max = pickle.load(handle)

inst = "ney"
feature = "db"
mini = min_max[inst]["min"][feature]
maxi = min_max[inst]["max"][feature]

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_magnitude = load_model("generator_sp_32_0_8_full").to(device)

src_dirs = {
    "gtr": ["dataset/gtr_1", "dataset/gtr_2"],
    "ney": ["dataset/ney_1", "dataset/ney_2"],
}

gtr_files = sorted([f for f in Path(src_dirs["gtr"][0]).glob("*.wav")])
ney_files = sorted([f for f in Path(src_dirs["ney"][0]).glob("*.wav")])

len_window = WINDOW_SAMPLE_LEN
len_overlap = OVERLAP
start_offset = len_window - len_overlap

for gtr_file in gtr_files:
    signal, _ = librosa.load(gtr_file, mono=True, sr=SR)
    i = 0
    dbs = []
    phases = []
    while True:
        start = i * start_offset
        end = start + len_window
        window = signal[start:end]
        # pad with trailing 0's if window is smaller than required length
        if len(window) < len_window:
            window = np.pad(window, (0, len_window - len(window)))

        # stft -> phase, magnitude & db
        stft = librosa.stft(window, n_fft=N_FFT, hop_length=HOP)
        magnitude = np.abs(stft)
        phase = np.angle(stft)
        phases.append(phase)
        db = librosa.amplitude_to_db(magnitude)
        # normalization
        db = (db - mini) / (maxi - mini)
        db = np.expand_dims(db, axis=0)
        dbs.append(db)

        if end >= len(signal):
            break
        i += 1

    # dbs = torch.from_numpy(np.array(dbs, dtype=np.float32))
    break